In [1]:
import numpy as np
import pandas as pd
import os
import json
import pickle
import torch   
import copy 

from sklearn.model_selection import train_test_split
from scipy.optimize import curve_fit
from sklearn.metrics import r2_score

import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

# Plotting parameters
plt.rcParams.update({
"text.usetex":True,
"font.family":"sans-serif",
"font.serif":["Computer Modern Roman"]})
plt.rcParams['xtick.labelsize'] = 12
plt.rcParams['ytick.labelsize'] = 15
plt.rcParams['axes.labelsize'] = 12
plt.rcParams['axes.titlesize'] = 12
plt.rcParams['xtick.major.size'] = 5
plt.rcParams['xtick.major.width'] = 1
plt.rcParams['xtick.minor.size'] = 3
plt.rcParams['xtick.minor.width'] = 1
plt.rcParams['ytick.major.size'] = 5
plt.rcParams['ytick.major.width'] = 1
plt.rcParams['ytick.minor.size'] = 3
plt.rcParams['ytick.minor.width'] = 1

# User defined files and classes
import sys
sys.path.append('/Users/maitreyeesharma/WORKSPACE/PostDoc/Chemistry/SPIRAL/codes/RL/ReLMM/scripts/')

In [2]:
## Main Function

# Reading the input json file with dataset filename and path information
with open('../inputs.json', "r") as f:
    input_dict = json.load(f)

run_folder = input_dict['RunFolder']
input_type = input_dict['InputType']
input_path = input_dict['InputPath']
input_file = input_dict['InputFile']
output_dir = input_dict['OutputDirectory']

# Create a new output directory if it does not exist
isExist = os.path.exists(output_dir)
if not isExist:
    os.makedirs(output_dir)
    print("The new directory is created!", output_dir)

lookup_df = pickle.load(open(input_path+input_file, 'rb'))
lookup_df['cation-electronegativity']=lookup_df['cation-electronegativity'].replace(1.9,2.33)

lookup_df.loc[lookup_df['cation'] == 'Ge', 'cation-electronegativity'] = 2.01
lookup_df.loc[lookup_df['cation'] == 'Sn', 'cation-electronegativity'] = 1.9
lookup_df.to_pickle("/Users/maitreyeesharma/WORKSPACE/PostDoc/Chemistry/SPIRAL/codes/RL/ReLMM/"+
                    "MHP_dataset/perovskites.pkl")  



In [3]:
lookup_df

,organic,anion,cation,hse06,gga,organic-homo,organic-lumo,organic-dipole,organic-atomization,organic-r_gyr,organic-total_mass,anion-electron_affinity,anion-ionization_energy,anion-mass,anion-electronegativity,cation-electron_affinity,cation-ionization_energy,cation-mass,cation-electronegativity
0,acetamidinium,F,Ge,5.1528,3.8145,-0.49721,-0.22400,1.3334,-1.725934,1.344149,59.092,3.401189,17.42282,18.998,4.0,1.232676,7.89940,72.63,2.01
1,acetamidinium,Cl,Ge,3.7624,2.8878,-0.49721,-0.22400,1.3334,-1.725934,1.344149,59.092,3.612724,12.96764,35.450,3.0,1.232676,7.89940,72.63,2.01
2,acetamidinium,Br,Ge,2.9861,2.2149,-0.49721,-0.22400,1.3334,-1.725934,1.344149,59.092,3.363588,11.81381,79.904,2.8,1.232676,7.89940,72.63,2.01
3,acetamidinium,I,Ge,2.3129,1.6986,-0.49721,-0.22400,1.3334,-1.725934,1.344149,59.092,3.059046,10.45126,126.900,2.5,1.232676,7.89940,72.63,2.01
4,acetamidinium,F,Sn,4.3173,3.2079,-0.49721,-0.22400,1.3334,-1.725934,1.344149,59.092,3.401189,17.42282,18.998,4.0,1.112070,7.34390,118.71,1.90
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
187,trimethylammonium,I,Sn,1.9374,1.3633,-0.58397,-0.18158,0.8595,-2.045617,1.393929,60.120,3.059046,10.45126,126.900,2.5,1.112070,7.34390,118.71,1.90
188,trimethylammonium,F,Pb,4.8213,3.4595,-0.58397,-0.18158,0.8595,-2.045617,1.393929,60.120,3.401189,17.42282,18.998,4.0,0.356743,7.41666,207.20,2.33
189,trimethylammonium,Cl,Pb,3.5090,2.5224,-0.58397,-0.18158,0.8595,-2.045617,1.393929,60.120,3.612724,12.96764,35.450,3.0,0.356743,7.41666,207.20,2.33
190,trimethylammonium,Br,Pb,2.9537,2.0366,-0.58397,-0.18158,0.8595,-2.045617,1.393929,60.120,3.363588,11.81381,79.904,2.8,0.356743,7.41666,207.20,2.33
